<a href="https://colab.research.google.com/github/AdamLeeIT/DATA715/blob/main/pubmed_etl_exercise_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Bio

In [5]:
from Bio import Entrez
from Bio import Medline
from tqdm import tqdm
from itertools import combinations
import json

In [7]:
# Change this email to your email address
Entrez.email = "user@email.com"

keyword = '"graph database"" AND 2025[pdat]'


result = Entrez.read(Entrez.esearch(db="pubmed", retmax=10, term=keyword))
print(
    "Total number of publications that contain the term {}: {}".format(
        keyword, result["Count"]
    )
)

# Fetch all ids
MAX_COUNT = result["Count"]
result = Entrez.read(
    Entrez.esearch(db="pubmed", retmax=result["Count"], term=keyword)
   #  Entrez.esearch(db="pubmed", retmax=10, term=keyword)

)

ids = result["IdList"]

batch_size = 100
batches = [ids[x: x + 100] for x in range(0, len(ids), batch_size)]

record_list = []
record_list2 = []

for batch in tqdm(batches):
    h = Entrez.efetch(db="pubmed", id=batch, rettype="medline", retmode="json")
    records = Medline.parse(h)
    record_list.extend(list(records))
print("Complete.")

Total number of publications that contain the term "graph database"" AND 2025[pdat]: 25


100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Complete.


In [8]:
print(json.dumps(record_list))

[{"PMID": "41425057", "OWN": "NLM", "STAT": "PubMed-not-MEDLINE", "DCOM": "20251222", "LR": "20251224", "IS": "2624-8212 (Electronic) 2624-8212 (Linking)", "VI": "8", "DP": "2025", "TI": "NatureKG: an ontology and knowledge graph for nature finance with a Text2Cypher application.", "PG": "1693843", "LID": "10.3389/frai.2025.1693843 [doi] 1693843", "AB": "INTRODUCTION: Nature finance involves complex, multi-dimensional challenges that require analytical frameworks to assess risks, impacts, dependencies, and systemic resilience. Existing financial systems lack structured tools to map dependencies between natural capital and financial assets. To address this, we introduce NatureKG, the first ontology and instantiated knowledge graph (KG) specifically tailored to nature finance, aiming to support financial institutions in assessing environmental risks, impacts, and dependencies systematically. METHODS: We designed a domain ontology grounded in ENCORE, the Science-Based Targets Network (SBT

In [10]:
# Writing the dictionary to a JSON file
with open(r'content/pubmed_search_graph_theory.json', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(record_list))

In [11]:
# Extract only PMID and OT fields, handling cases where 'OT' might not exist
extracted_data = [{"PMID": entry["PMID"], "OT": entry.get("OT", [])} for entry in record_list]

# Display the extracted data
extracted_data

[{'PMID': '41425057',
  'OT': ['Neo4j',
   'Text2Cypher',
   'cypher query',
   'graph database',
   'knowledge graphs',
   'large language models',
   'nature finance',
   'question answering']},
 {'PMID': '41369777', 'OT': []},
 {'PMID': '41292650',
  'OT': ['BERT embeddings',
   'Case reports',
   'PubMed',
   'data mining',
   'evidence based medicine',
   'graph database',
   'network analysis',
   'semantic similarity']},
 {'PMID': '41252800',
  'OT': ['Cancer gene networks',
   'Gene pathway fingerprint',
   'Identifying cancer drug targets',
   'Matrix of genes and reactions',
   'Pathways shared in cancer and comorbid Mendelian Diseases',
   'Targeted cancer therapy combinations']},
 {'PMID': '41173963',
  'OT': ['Gear shaving tooth profile cutting depth error',
   'Knowledge graph',
   'Shaving-induced mid-profile concavity error',
   'Siloization issue',
   'Tooth profile error']},
 {'PMID': '41139924', 'OT': ['EGNF', 'biomarker', 'graph neural network']},
 {'PMID': '4103066

In [12]:
# Writing the dictionary to a JSON file
with open(r'content/pubmed_search_graph_theory_kws.json', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(extracted_data))

In [14]:
import json

# File paths
input_file = "/content/pubmed_search_graph_theory_kws.json"
output_file = "/content/pubmed_inserts.sql"

# Load JSON
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

with open(output_file, "w", encoding="utf-8") as f:
    f.write("INSERT INTO pubmed_raw (PMID, OT) VALUES\n")

    values = []

    for record in data:
        pmid = str(record.get("PMID", "")).strip()
        ot = record.get("OT", [])

        if pmid:
            if not isinstance(ot, list):
                ot = [ot]

            # Convert list to JSON string
            ot_json = json.dumps(ot, ensure_ascii=False)

            # Escape single quotes for SQL safety
            ot_json = ot_json.replace("'", "''")

            values.append(f"('{pmid}', CAST('{ot_json}' AS JSON))")

    f.write(",\n".join(values))
    f.write(";\n")

print("Finished. SQL file saved to:", output_file)

Finished. SQL file saved to: /content/pubmed_inserts.sql
